# 이미지 설명 요청하기

## 인터넷에 있는 이미지 사진 설명 요청하기
- 원하는 이미지 링크 사용
- 단, API 호출을 통해 서버에 접근 가능한 이미지 링크를 사용하기

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPEN_API_KEY")  # 환경 변수에서 API 키를 가져오기

client = OpenAI(api_key=api_key)  # 오픈AI 클라이언트의 인스턴스 생성

messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "이 이미지에 대해 설명해주세요."},
            {
                "type": "image_url",
                "image_url": {
                    "url": "https://recipe1.ezmember.co.kr/cache/recipe/2016/06/05/1ba7e34cf0daf694f09a3a9539ebdb161.jpg",
                },
            },
        ],
    }
]

response = client.chat.completions.create(
    model="gpt-4o",  # 응답 생성에 사용할 모델 지정
    messages=messages # 대화 기록을 입력으로 전달
)

response


ChatCompletion(id='chatcmpl-CQP6rnFlFmaqn0m3klKoaWwwH6nP0', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='이 이미지는 오므라이스입니다. 오므라이스는 계란으로 덮은 밥 요리로, 일반적으로 밥 위에 계란을 얹어 만들며 주로 케첩을 뿌려 먹습니다. 사진 속 오므라이스는 가늘게 케첩을 뿌린 상태이며, 안에는 햄과 채소가 함께 볶아진 밥이 들어 있습니다. 접시에는 오므라이스 외에도 브로콜리와 얇게 썬 배가 함께 담겨 있습니다.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[]))], created=1760409085, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_cbf1785567', usage=CompletionUsage(completion_tokens=121, prompt_tokens=779, total_tokens=900, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

## 내가 가진 이미지로 설명 요청하기

### base64 인코딩
- 바이너리 데이터 HTTP Request 전송을 위해 인코딩 진행

In [2]:
import base64

# Function to encode the image
def encode_image(path):
    with open(path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
    
image_path = "./data/mart.jpg"

# 이미지를 base64로 인코딩
base64_image = encode_image(image_path)

print(base64_image[0:100])

/9j/4AAUSkZJRgABAQEBLAEsAABBTVBG/+EJxEV4aWYAAE1NACoAAAAIAA0BDwACAAAABgAAAKoBEAACAAAACgAAALABEgADAAAA


### GPT API 호출 및 결과 출력

In [3]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "이 이미지에 대해 설명해주세요."},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}",
                },
            },
        ],
    }
]

response = client.chat.completions.create(
    model="gpt-4o",  # 응답 생성에 사용할 모델 지정
    messages=messages # 대화 기록을 입력으로 전달
)

response.choices[0].message.content

'이미지에는 슈퍼마켓이나 편의점의 한 진열대가 보입니다. 선반 위에는 다양한 즉석 라면 제품들이 진열되어 있습니다. 오른쪽에는 컵라면이 여러 개 쌓여 있으며, 왼쪽에는 봉지라면이 박스에 담겨 있습니다. 아래쪽에는 쌀국수로 보이는 제품들이 포장된 상태로 진열되어 있습니다. 바닥과 진열대는 타일과 메탈 소재로 보이며 깔끔하게 정리되어 있습니다.'

### 여러 이미지 비교 분석 요청

In [4]:
red_dumpling_base64 = encode_image("data/red_dumpling.jpeg")
dumpling_soup_base64 = encode_image("data/dumpling_soup.jpeg")

messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "두 음식의 차이점을 설명해주세요."},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{red_dumpling_base64}",
                },
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{dumpling_soup_base64}",
                },
            },
        ],
    }
]

response = client.chat.completions.create(
    model="gpt-4o",  # 응답 생성에 사용할 모델 지정
    messages=messages # 대화 기록을 입력으로 전달
)

response.choices[0].message.content

'첫 번째 음식은 빨간 기름 소스에 담긴 만두입니다. 매콤한 소스가 특징으로, 일반적으로 중국식인 훠궈나 라면에 자주 사용되는 향신료가 들어갑니다.\n\n두 번째 음식은 국물 요리로, 만두가 맑은 육수에 담겨 있습니다. 국물에는 계란지단 같은 고명이 추가되어 담백한 맛이 강조됩니다. 이는 한국이나 중국의 전통적인 만두국 스타일로 볼 수 있습니다.\n\n두 음식 모두 만두를 사용하지만, 첫 번째는 매콤한 소스, 두 번째는 담백한 국물이 차이를 이루고 있습니다.'

## GPT 비전의 한계 알아보기

In [5]:
oecd_rnd_2021_base64 = encode_image("./data/oecd_rnd_2021_large.png")
oecd_rnd_2022_base64 = encode_image("./data/oecd_rnd_2022_large.png")

messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "첫번째는 2021년 데이터이고, 두번째는 2022년 데이터입니다. 이 데이터에 대해 설명해주세요. 어떤 변화가 있었나요? 한국 중심으로 설명해주세요."},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{oecd_rnd_2021_base64}",
                },
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{oecd_rnd_2022_base64}",
                },
            },
        ],
    }
]

response = client.chat.completions.create(
    model="gpt-4o",  # 응답 생성에 사용할 모델 지정
    messages=messages # 대화 기록을 입력으로 전달
)

response.choices[0].message.content


'첫 번째는 2021년 데이터, 두 번째는 2022년 데이터입니다. \n\n1. **연구개발비**:\n   - **한국의 연구개발비**: \n     - 2021년에는 약 89,282백만 달러에서 2022년에는 약 91,013백만 달러로 증가했습니다.\n   - **미국, 중국, 일본** 등의 주요 국가들도 연구개발비가 전반적으로 증가했습니다.\n\n2. **GDP 대비 연구개발비 비중**:\n   - **한국의 비중**:\n     - 2021년에는 4.93%에서 2022년에는 4.95%로 소폭 증가했습니다.\n   - **다른 국가**: \n     - 스위스와 이스라엘은 여전히 높은 비중을 유지하고 있으며, 일부 국가에서는 소폭의 변화가 있습니다.\n\n3. **순위 및 변화**:\n   - 한국은 연구개발비 규모 면에서 큰 변화 없이 상위권을 유지하고 있으며, GDP 대비 비중 면에서도 상위권을 유지하고 있습니다.\n\n전체적으로, 한국은 연구개발 투자에서 큰 증가세를 보이고 있으며 글로벌 경쟁력도 유지하고 있습니다.'

In [6]:
oecd_rnd_2021_base64 = encode_image("./data/oecd_rnd_2021_medium.png")
oecd_rnd_2022_base64 = encode_image("./data/oecd_rnd_2022.png")

messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "첫번째는 2021년 데이터이고, 두번째는 2022년 데이터입니다. 이 데이터에 대해 설명해주세요. 어떤 변화가 있었나요? 한국 중심으로 설명해주세요."},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{oecd_rnd_2021_base64}",
                },
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{oecd_rnd_2022_base64}",
                },
            },
        ],
    }
]

response = client.chat.completions.create(
    model="gpt-4o",  # 응답 생성에 사용할 모델 지정
    messages=messages # 대화 기록을 입력으로 전달
)

response.choices[0].message.content

'두 이미지에는 각각 2021년과 2022년 OECD 국가들의 연구개발비와 GDP 대비 연구개발비 비중이 나타나 있습니다. 한국을 중심으로 주요 변화를 살펴보면 다음과 같습니다:\n\n1. **연구개발비(백만 US 달러)**:\n   - 한국의 연구개발비는 2021년 91,013백만 달러에서 2022년 87,225백만 달러로 감소했습니다.\n\n2. **GDP 대비 연구개발비 비중(%)**:\n   - 2021년에는 4.93%로 높은 비중을 차지했으나, 2022년에는 5.21%로 소폭 증가했습니다.\n\n이를 통해 한국은 연구비용에서는 감소했지만, GDP 대비 비중은 증가했다는 것을 알 수 있습니다. 이는 경제 성장에 비해 연구개발 투자가 여전히 높은 수준을 유지하고 있음을 시사합니다. \n\n또한 다른 주요 국가들, 예를 들어 미국과 중국에서도 연구개발비가 증가했음을 알 수 있습니다.'